### Note: we created elastic net functions using both ADMM and CD optimizations

In [5]:
import numpy as np
import time
import pandas as pd
from sklearn import linear_model

### Function definition: Elastic Net with ADMM

In [6]:
def soft_thresh(x, lambda_val):
    #all should be shape 400* 1
    return np.sign(x)* np.maximum(np.abs(x) - lambda_val, np.zeros(x.shape))

In [7]:
def elasticNet(X, y, tau, tau2, max_iterations=1000, tol=1e-4):
    XX = X.T.dot(X)
    Xy = X.T.dot(y) #400*1
    
    n = X.shape[0]
    p = X.shape[1] #number of parameters
    lambdas = np.zeros((p,1))
    max_rho = 5
    rho = 4

    z0, z, beta0, beta = [np.zeros((p,1))] * 4
    sinv = np.linalg.inv(XX/n + rho* np.eye(p))
    #print(sinv.shape)

    for i in range(max_iterations):
        # update beta
        beta = sinv.dot(Xy/n + (rho * z).reshape(-1,1) - lambdas.reshape(-1,1)) #400*1
        #print(beta.shape) #400*1
        
        # update z
        z = soft_thresh(beta + lambdas.reshape(-1,1)/rho, tau /rho) * (rho /(rho + tau2))
        #print(z.shape) #this is wrong 400 *400 should be 400*1

        # update lambda
        lambdas = lambdas + rho* (beta - z)

        change = max(np.linalg.norm(beta - beta0), np.linalg.norm(z - z0))
        if change < tol or i + 1 > max_iterations:
            break
        beta0, z0 = beta, z
    return z

### Data reading and testing single setting ($\alpha=1, \lambda=1$)

In [8]:
# Data reading
X = pd.read_csv('X.csv', sep=',',header=None)
print(X.shape) #50*400, 400 variables 
X_val = X.values #numpy 
y = pd.read_csv('y.csv', sep=',',header=None)
print(y.shape) #50*1
y_val = y.values 

(50, 400)
(50, 1)


In [9]:
# testing one result only
start_time = time.time()
re = elasticNet(X, y, tau = 1, tau2= 0)
print(time.time() - start_time)

0.21530890464782715


In [10]:
# check if answers match
# glmnet
regr = linear_model.ElasticNet(l1_ratio=1, fit_intercept=False, alpha=1, random_state =1234)
start_time = time.time()
fit = regr.fit(X, y)
print(time.time() - start_time)

0.0036840438842773438


In [11]:
re_glmnet = fit.coef_

beta_mat = [re.reshape(400,),  re_glmnet]
#print(beta_mat)

In [12]:
# objective function value
def obj(beta, X, y, tau, tau2):
    return  np.linalg.norm(y - X.dot(beta.reshape(-1,1))) **2/100 + tau * np.linalg.norm(beta.reshape(-1,1), ord=1) + tau2 *(np.linalg.norm(beta.reshape(-1,1))**2)

result = [obj(b, X, y, tau = 1, tau2 = 0) for b in beta_mat]
print(result) #close enough

[26.01694539746661, 26.016945325528923]


### Actual experiments. Setting 1: alpha = 0.95, lambda  = 0.01, 0.1, 1, 10

In [13]:
betas = []
alpha = 0.95
for labd in [0.01, 0.1, 1, 10]:
    tau = alpha * labd
    tau2 = 0.5 *labd* (1-alpha)
    re = elasticNet(X, y, tau = tau, tau2= tau2)
    betas.append(re)
    
betas = np.stack(betas, axis =1)
betas = np.squeeze(betas, axis = 2)
print(betas.shape)
np.save('setting1_betas', betas)

(400, 4)


In [14]:
betas

array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       ...,
       [ 0.        , -0.        ,  0.        ,  0.        ],
       [ 0.01814772,  0.        ,  0.        ,  0.        ],
       [-0.        ,  0.        , -0.        , -0.        ]])

### Actual experiments.  Setting 2: alpha = 1, lambda  = 0.01, 0.1, 1, 10

In [15]:
betas = []
alpha = 1
for labd in [0.01, 0.1, 1, 10]:
    tau = alpha * labd
    tau2 = 0.5 *labd* (1-alpha)
    re = elasticNet(X, y, tau = tau, tau2= tau2)
    betas.append(re)
    
betas = np.stack(betas, axis =1)
betas = np.squeeze(betas, axis = 2)
print(betas.shape)
np.save('setting2_betas', betas)

(400, 4)


In [16]:
betas

array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       ...,
       [ 0.        , -0.        ,  0.        ,  0.        ],
       [ 0.00182055,  0.        ,  0.        ,  0.        ],
       [-0.        ,  0.        , -0.        , -0.        ]])

### Function definition: Elastic Net with CD

In [17]:
def soft_thresholding(x, a):
    # assuming a is non-negative
    a = np.array(a)
    x = np.array(x)
    result = np.zeros(x.size)
    result[x > a] = x[x > a] - a 
    result[x < -a] = x[x < -a] + a
    return result    #Will remain zero if the abs val of x is less than or equal to a

In [18]:
def elastic_net_cd(X, y, beta, alpha, lambda_val, tol=1e-4, max_iter=1000, quiet=False):

    obj = np.zeros(max_iter + 1)
    beta_list = [np.copy(beta)]

    j = 0
    for j in range(max_iter):
        for k in range(beta.size):
            r = y - np.delete(X, k, axis=1).dot(np.delete(beta, k))

            beta[k] = (1 / (np.linalg.norm(X[:, k]) ** 2)) \
                      * soft_thresholding([r.T.dot(X[:, k])], [len(y) * lambda_val])
        beta_list.append(np.copy(beta))
        obj[j] = (1/2.) * (1./len(y)) * np.linalg.norm(y - X.dot(beta)) ** 2 + lambda_val * (alpha * np.sum(np.abs(beta)) + (1-alpha)  * beta.dot(beta) / 2.)

        if np.linalg.norm(beta_list[j] - beta) < tol:
            break

    return {
        'obj': obj[:j],
        'beta': beta
    }

In [19]:
##Redefining data for use with new function
df_X = pd.read_csv('X.csv', header=None)
df_y = pd.read_csv('y.csv', header=None)
X = df_X.values
y = df_y.values
y = y.reshape(50)
p = X.shape[1]

### Testing single setting ($\alpha=0.95, \lambda=0.1$)

In [20]:
start_time = time.time()
re = elastic_net_cd(X, y, np.zeros(p), 0.95, 0.1)
print(time.time() - start_time)

6.152191877365112


In [21]:
regr = linear_model.ElasticNet(l1_ratio=0.95, fit_intercept=False, alpha=0.1, random_state =1234)
start_time = time.time()
fit = regr.fit(X, y)
print(time.time() - start_time)

0.008445978164672852


In [22]:
# objective function value
def obj(beta, X, y, alpha, lambda_val):
    return  (1/2.) * (1./len(y)) * np.linalg.norm(y - X.dot(beta)) ** 2 + lambda_val * (alpha * np.sum(np.abs(beta)) + (1-alpha) * beta.dot(beta) / 2.)
re_glmnet = fit.coef_
sklearn_result = obj(re_glmnet, X, y, 0.95, 0.1)
print(sklearn_result, min(re['obj'])) ##slightly off but close

3.4048228731260677 3.4118205832606345


### Actual experiments. Setting 1: alpha = 0.95, lambda  = 0.01, 0.1, 1, 10

In [23]:
en_out = np.zeros((p,4))
lambdas = [0.01, 0.1, 1., 10.]
j=0
for i in lambdas:
    re = elastic_net_cd(X, y, np.zeros(p), 0.95, i)
    en_out[:,j] = re['beta'] 
    j=j+1
en_betas = pd.DataFrame({'lambda=0.01': en_out[:, 0], 'lambda=0.1': en_out[:, 1], 'lambda=1': en_out[:, 2], 'lambda=10': en_out[:, 3]})

en_betas

,lambda=0.01,lambda=0.1,lambda=1,lambda=10
0,0.000000,0.000000,0.000000,0.0
1,0.000000,0.000000,0.000000,0.0
2,0.178000,0.000000,0.000000,0.0
3,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.0
5,0.000000,0.000000,0.000000,0.0
6,0.000000,0.000000,0.000000,0.0
7,0.000000,0.000000,0.000000,0.0
8,0.000000,0.000000,0.000000,0.0
9,0.000000,0.000000,0.000000,0.0


In [26]:
## To find non-zero coefficients
print(np.argwhere(en_out[:,2]>0.0))

[[ 15]
 [ 17]
 [ 82]
 [141]
 [300]
 [301]
 [302]
 [303]
 [304]]


## Actual experiments. Setting 2 (LASSO): alpha=1, lambda=0.01,0.1,1,10

In [25]:
lasso_out = np.zeros((p,4))
j=0
for i in lambdas:
    re = elastic_net_cd(X, y, np.zeros(p), 1., i)
    lasso_out[:,j] = re['beta'] 
    j=j+1
lasso_betas = pd.DataFrame({'lambda=0.01': lasso_out[:, 0], 'lambda=0.1': lasso_out[:, 1], 'lambda=1': lasso_out[:, 2], 'lambda=10': lasso_out[:, 3]})

lasso_betas

,lambda=0.01,lambda=0.1,lambda=1,lambda=10
0,0.000000,0.000000,0.000000,0.0
1,0.000000,0.000000,0.000000,0.0
2,0.178000,0.000000,0.000000,0.0
3,0.000000,0.000000,0.000000,0.0
4,0.000000,0.000000,0.000000,0.0
5,0.000000,0.000000,0.000000,0.0
6,0.000000,0.000000,0.000000,0.0
7,0.000000,0.000000,0.000000,0.0
8,0.000000,0.000000,0.000000,0.0
9,0.000000,0.000000,0.000000,0.0
